In [1]:
!pip install selenium webdriver-manager


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\NGOCANH\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
from openpyxl import Workbook

def is_vietnamese(text):
    """
    Kiểm tra xem một chuỗi có chứa ký tự tiếng Việt có dấu hay không.
    Regex này tập trung vào các ký tự đặc trưng của tiếng Việt.
    """
    return bool(re.search(r'[\u00C0-\u017F\u1E00-\u1EFF]', text))

def scroll_to_bottom(driver, scroll_pause_time=2, max_scroll_attempts=100):
    """
    Cuộn xuống cuối trang để tải tất cả các bình luận chính.
    Dừng lại khi chiều cao trang không thay đổi nữa.
    """
    print("\nBắt đầu cuộn trang để tải bình luận chính...")
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    attempts = 0
    
    for i in range(max_scroll_attempts):
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        
        if new_height == last_height:
            attempts += 1
            if attempts >= 3: # Thử 3 lần liên tiếp không thay đổi thì dừng
                print("Đã cuộn đến cuối trang.")
                break
        else:
            attempts = 0 # Reset bộ đếm nếu có nội dung mới
        
        last_height = new_height
        if (i + 1) % 5 == 0:
            print(f"Đã cuộn {i + 1} lần...")
    print("Hoàn tất cuộn trang.")

def expand_all_replies(driver, wait):
    """
    Tìm và nhấp vào tất cả các nút 'Xem câu trả lời' để hiển thị các bình luận lồng nhau.
    """
    print("\nBắt đầu mở rộng các bình luận trả lời...")
    try:
        # Tìm tất cả các nút mở rộng trả lời
        reply_buttons = wait.until(EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, "#more-replies ytd-button-renderer")
        ))
        
        clicked_count = 0
        for button in reply_buttons:
            try:
                # Dùng JavaScript để click, đáng tin cậy hơn
                driver.execute_script("arguments[0].scrollIntoView(true);", button)
                time.sleep(20) # Chờ một chút để đảm bảo nút có thể click
                driver.execute_script("arguments[0].click();", button)
                clicked_count += 1
                if clicked_count % 10 == 0:
                    print(f"Đã mở rộng {clicked_count} nhóm trả lời...")
                time.sleep(1) # Chờ để các trả lời được tải
            except Exception as e:
                # Bỏ qua nếu nút không còn tồn tại hoặc không thể click
                pass
        print(f"Hoàn tất! Đã mở rộng tổng cộng {clicked_count} nhóm trả lời.")
    except TimeoutException:
        print("Không tìm thấy nút 'Xem câu trả lời' nào, hoặc đã mở rộng hết.")

def main():
    # --- Cấu hình ---
    options = Options()
    options.add_argument("--start-maximized")
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--no-sandbox')
    options.add_argument('log-level=3') # Giảm bớt log không cần thiết trên console

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    wait = WebDriverWait(driver, 15)

    # --- URL Video và File Output ---
    video_url = "https://www.youtube.com/watch?v=IdlUtQvpRn8"
    output_file = 'D:/youtube_comments_full_vietnamese.xlsx'

    # --- Bắt đầu quá trình ---
    print(f"Đang truy cập video: {video_url}")
    driver.get(video_url)

    # --- ĐIỂM CẢI TIẾN QUAN TRỌNG ---
    # Cuộn nhẹ xuống để kích hoạt việc tải bình luận
    print("Cuộn nhẹ xuống để kích hoạt tải bình luận...")
    driver.execute_script("window.scrollTo(0, 500);")
    time.sleep(20)

    # Chờ cho đến khi BÌNH LUẬN ĐẦU TIÊN xuất hiện
    try:
        # ytd-comment-thread-renderer là thẻ chứa một bình luận hoàn chỉnh
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "ytd-comment-thread-renderer")))
        print("Đã tìm thấy bình luận đầu tiên. Bắt đầu quá trình chính...")
    except TimeoutException:
        print("Không tìm thấy bình luận nào sau 15 giây. Có thể video đã tắt bình luận.")
        driver.quit()
        return

    # Cuộn để tải các bình luận chính
    scroll_to_bottom(driver)

    # Mở rộng tất cả các bình luận trả lời
    expand_all_replies(driver, wait)
    
    print("\nĐang chờ tất cả bình luận được tải hoàn tất...")
    time.sleep(20) # Chờ thêm một chút để đảm bảo mọi thứ đã ổn định

    # --- Thu thập dữ liệu ---
    print("Bắt đầu thu thập và lọc bình luận...")
    all_comments_data = []
    index = 1
    
    comment_threads = driver.find_elements(By.CSS_SELECTOR, "ytd-comment-thread-renderer")
    
    if not comment_threads:
        print("⚠️ Không tìm thấy bình luận nào. Kết thúc.")
        driver.quit()
        return

    print(f"🔎 Tìm thấy {len(comment_threads)} luồng bình luận. Đang xử lý...")

    for thread in comment_threads:
        try:
            main_comment_element = thread.find_element(By.CSS_SELECTOR, "#comment #content-text")
            main_comment_text = main_comment_element.text.strip()
            if is_vietnamese(main_comment_text):
                all_comments_data.append([index, main_comment_text, "Bình luận chính"])
                index += 1

            reply_elements = thread.find_elements(By.CSS_SELECTOR, "#replies #content-text")
            for reply_element in reply_elements:
                reply_text = reply_element.text.strip()
                if is_vietnamese(reply_text):
                    all_comments_data.append([index, reply_text, "Bình luận trả lời"])
                    index += 1
        except StaleElementReferenceException:
            continue
        except Exception as e:
            print(f"Lỗi khi xử lý một luồng bình luận: {e}")
            continue

    driver.quit()

    # --- Lưu vào file Excel ---
    if not all_comments_data:
        print("Không có bình luận tiếng Việt nào được tìm thấy để lưu.")
        return

    wb = Workbook()
    ws = wb.active
    ws.title = "YouTube Comments (Vietnamese)"
    ws.append(["Index", "Comment", "Type"])

    for comment_data in all_comments_data:
        ws.append(comment_data)

    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column].width = adjusted_width

    wb.save(output_file)
    print(f"\n✅ Thành công! Đã lưu {len(all_comments_data)} bình luận tiếng Việt vào file '{output_file}'")

if __name__ == "__main__":
    main()

Đang truy cập video: https://www.youtube.com/watch?v=0P2FWqek8sc&t=426s
Cuộn nhẹ xuống để kích hoạt tải bình luận...
Đã tìm thấy bình luận đầu tiên. Bắt đầu quá trình chính...

Bắt đầu cuộn trang để tải bình luận chính...
Đã cuộn 5 lần...
Đã cuộn 10 lần...
Đã cuộn 15 lần...
Đã cuộn 20 lần...
Đã cuộn 25 lần...
Đã cuộn 30 lần...
Đã cuộn 35 lần...
Đã cuộn 40 lần...
Đã cuộn 45 lần...
Đã cuộn 50 lần...
Đã cuộn đến cuối trang.
Hoàn tất cuộn trang.

Bắt đầu mở rộng các bình luận trả lời...
Không tìm thấy nút 'Xem câu trả lời' nào, hoặc đã mở rộng hết.

Đang chờ tất cả bình luận được tải hoàn tất...
Bắt đầu thu thập và lọc bình luận...
🔎 Tìm thấy 635 luồng bình luận. Đang xử lý...

✅ Thành công! Đã lưu 556 bình luận tiếng Việt vào file 'D:/youtube_comments_full_vietnamese.xlsx'
